In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
import os, torch

os.environ["CUDA_VISIBLE_DEVICES"] = "7"
# device0 = "cuda:0" if torch.cuda.is_available() else "cpu"
device7 = "cuda:0" if torch.cuda.is_available() else "cpu"
OCCUPY_CUDA_7 = torch.randn(1, 1, device=device7)


In [5]:

import csv
sample_path = 'reference_data/sample_pfda/ValidationDataset-sampledata.csv'
rectified_path = 'data/rectified_dataset_fog_release.csv'
with open(sample_path, mode ='r')as file:
    csvFile = csv.reader(file)
    count = 0
    for lines in csvFile:
        if count == 2:
                break
        print(lines)
        count+=1
        

['Time', 'GeneralEvent', 'ClinicalEvent', 'L Foot Contact', 'R Foot Contact', 'L Foot Pressure', 'R Foot Pressure', 'Walkway_X', 'Walkway_Y', 'WalkwayPressureLevel', 'WalkwayFoot', 'LowerBack_Acc_X', 'LowerBack_Acc_Y', 'LowerBack_Acc_Z', 'LowerBack_FreeAcc_E', 'LowerBack_FreeAcc_N', 'LowerBack_FreeAcc_U', 'LowerBack_Gyr_X', 'LowerBack_Gyr_Y', 'LowerBack_Gyr_Z', 'LowerBack_Mag_X', 'LowerBack_Mag_Y', 'LowerBack_Mag_Z', 'LowerBack_VelInc_X', 'LowerBack_VelInc_Y', 'LowerBack_VelInc_Z', 'LowerBack_OriInc_q0', 'LowerBack_OriInc_q1', 'LowerBack_OriInc_q2', 'LowerBack_OriInc_q3', 'LowerBack_Roll', 'LowerBack_Pitch', 'LowerBack_Yaw', 'R_Wrist_Acc_X', 'R_Wrist_Acc_Y', 'R_Wrist_Acc_Z', 'R_Wrist_FreeAcc_E', 'R_Wrist_FreeAcc_N', 'R_Wrist_FreeAcc_U', 'R_Wrist_Gyr_X', 'R_Wrist_Gyr_Y', 'R_Wrist_Gyr_Z', 'R_Wrist_Mag_X', 'R_Wrist_Mag_Y', 'R_Wrist_Mag_Z', 'R_Wrist_VelInc_X', 'R_Wrist_VelInc_Y', 'R_Wrist_VelInc_Z', 'R_Wrist_OriInc_q0', 'R_Wrist_OriInc_q1', 'R_Wrist_OriInc_q2', 'R_Wrist_OriInc_q3', 'R_Wris

In [10]:
data_folder = os.listdir('data')
rectified_csv_files = [file for file in data_folder if file.startswith('rec') \
                                                        and file.endswith('.csv')]
print(rectified_csv_files)
rectified_csv_files = sorted(rectified_csv_files, key=lambda x: (int(x[:-4].split('_')[1])))
rectified_csv_files

['rec_1.csv', 'rec_2.csv', 'rec_11.csv']


['rec_1.csv', 'rec_2.csv', 'rec_11.csv']

In [1]:
data = [
    ['header1', 0, 2, 1, 4],
    ['header2', 3, 5, 6]
]
print(data)
transposed_data = list(zip(*data))
transposed_data

[['header1', 0, 2, 1, 4], ['header2', 3, 5, 6]]


[('header1', 'header2'), (0, 3), (2, 5), (1, 6)]

In [9]:
import pandas as pd
file_path = "data/kaggle_pd_data/train/tdcsfog/0df04e8431.csv"
df = pd.read_csv(file_path)
print(df.columns)
# Count non-zero elements in the specified column
non_zero_count = (df["Turn"] != 0).sum()
non_zero_count


Index(['Time', 'AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn', 'Walking'], dtype='object')


251

In [33]:
a = torch.randint(10, size=(50000,3))
def get_blocks(series):
    series = series.clone()
    # series = series[columns]
    
    block_count = math.ceil(series.shape[0] / CFG['block_size'])
    
    padding = block_count * CFG['block_size'] - series.shape[0]
    padding = torch.zeros(size=(padding, series.shape[1]))

    series = torch.cat([series, padding], dim=0)
    
    block_begins = list(range(0, series.shape[0], CFG['block_stride']))
    block_begins = [x for x in block_begins if x + CFG['block_size'] <= series.shape[0]]
    
    blocks = []
    for begin in block_begins:
        values = series[begin:begin + CFG['block_size']]
        blocks.append({'begin': begin,
                       'end': begin + CFG['block_size'],
                       'values': values})
    
    return blocks

blocks = get_blocks(a)
blocks[0]['values'].shape

torch.Size([15552, 3])

# Transformer + LSTM

In [1]:
import torch, math
import numpy as np
import os
import math
import random
import warnings
import numpy as np 
import pandas as pd

import matplotlib.pyplot as plt
import scipy
import torch.nn as nn

from tqdm import tqdm
from itertools import cycle
from joblib import Parallel, delayed
from sklearn.metrics import average_precision_score

device7 = "cuda:7" if torch.cuda.is_available() else "cpu"
OCCUPY_CUDA_7 = torch.randn(1, 1, device=device7)


In [2]:
print(OCCUPY_CUDA_7.device)

cuda:7


In [5]:
CFG = {
    'block_size': 15552, 
    'block_stride': 15552//16, # 972
    'patch_size': 18, 

    'fog_model_input_dim': 18*3,
    'fog_model_dim': 320,
    'fog_model_num_heads': 8, #6,
    'fog_model_num_encoder_layers': 5,
    'fog_model_num_lstm_layers': 2,
    'fog_model_first_dropout': 0.1,
    'fog_model_encoder_dropout': 0.1,
    'fog_model_mha_dropout': 0.0,
}

assert CFG['block_size'] % CFG['patch_size'] == 0  # 864
assert CFG['block_size'] % CFG['block_stride'] == 0  # 972

'''
Train and inference batch size

'''

GPU_BATCH_SIZE = 32


In [22]:
'''
The transformer encoder layer
'''

class EncoderLayer(nn.Module):
    def __init__(self, cfg):
        super(EncoderLayer, self).__init__()
        
        self.mha = nn.MultiheadAttention(embed_dim=cfg['fog_model_dim'], 
                                         num_heads=cfg['fog_model_num_heads'], 
                                         dropout=cfg['fog_model_mha_dropout'])
        
        self.layernorm = nn.LayerNorm(cfg['fog_model_dim'])
        
        self.seq = nn.Sequential(
            nn.Linear(cfg['fog_model_dim'], cfg['fog_model_dim']),
            nn.ReLU(),
            nn.Dropout(cfg['fog_model_encoder_dropout']),
            nn.Linear(cfg['fog_model_dim'], cfg['fog_model_dim']),
            nn.Dropout(cfg['fog_model_encoder_dropout'])
        )
        
    def forward(self, x):
        attn_output, _ = self.mha(query=x, key=x, value=x)
        x = x + attn_output
        x = self.layernorm(x)
        x = x + self.seq(x)
        x = self.layernorm(x)
        return x

# Example usage
encoder_layer = EncoderLayer(CFG)
input_tensor = torch.rand(32, 864, 320)
output = encoder_layer(input_tensor)
print(output.shape)
    


torch.Size([32, 864, 320])


In [18]:
'''
FOGEncoder is a combination of transformer encoder (D=320, H=6, L=5) and two BidirectionalLSTM layers

'''

class FOGEncoder(nn.Module):
    def __init__(self, cfg):
        super(FOGEncoder, self).__init__()
    
        self.first_linear = nn.Linear(cfg['fog_model_input_dim'], cfg['fog_model_dim'])
        
        self.first_dropout = nn.Dropout(cfg['fog_model_first_dropout'])
        
        self.enc_layers = nn.Sequential()
        for _ in range(cfg['fog_model_num_encoder_layers']):
            self.enc_layers.append(EncoderLayer(cfg))
            
        self.lstm_layers = nn.LSTM(cfg['fog_model_dim'], cfg['fog_model_dim'], 
                                   num_layers=cfg['fog_model_num_lstm_layers'],
                                   batch_first=True, bidirectional=True)
        
        self.sequence_len = cfg['block_size'] // cfg['patch_size'] # 864
        self.pos_encoding = nn.Parameter(torch.randn(1, self.sequence_len,
                                                     cfg['fog_model_dim']) * 0.02, 
                                         requires_grad=True)
        
    def forward(self, x, training=True):
        # x: (B, S, P*3), Example shape (4, 864, 54)
        batch_size = x.size(0)
        x = x / 25.0  # Normalization attempt in the segment [-1, 1]
        x = self.first_linear(x)  # (batch_size, sequence_len, fog_model_dim)

        if training:  # augmentation by randomly roll of the position encoding tensor
            shifts = torch.randint(low=-self.sequence_len, high=0, size=(batch_size,))
            random_pos_encoding = torch.cat([torch.roll(self.pos_encoding, 
                                                        shifts=s.item(), 
                                                        dims=1) \
                                             for s in shifts], dim=0)
            x = x + random_pos_encoding
        else:  # without augmentation
            x = x + self.pos_encoding.repeat(batch_size, 1, 1)
        
        x = self.first_dropout(x)
        
        x = self.enc_layers(x) # (B,S,D)
        
        x, _ = self.lstm_layers(x) # (B,S,2*D)

        return x
    
encoder = FOGEncoder(CFG)
encoder.to(device7)
input_tensor = torch.rand(32, 864, 54)  # (batch_size, sequence_length, input_dim)
input_tensor = input_tensor.to(device7)
output = encoder(input_tensor)
print(output.shape)
    

torch.Size([32, 864, 640])


In [25]:
class FOGModel(nn.Module):
    def __init__(self, cfg):
        super(FOGModel, self).__init__()
        
        self.encoder = FOGEncoder(cfg)
        self.last_linear = nn.Linear(cfg['fog_model_dim'] * 2, 1)
        
    def forward(self, x):  
        # x: (B,S,input_D), e.g. (4, 864, 54)
        x = self.encoder(x)  # (B,S,D*2), e.g. (4, 864, 640)
        x = self.last_linear(x)  # (B,S,3), e.g. (4, 864, 3)
        x = torch.sigmoid(x)  # Sigmoid activation
        return x
    
model = FOGModel(CFG)
model.to(device7)
input_tensor = torch.rand(4, 864, 54)  # (batch_size, sequence_len, input_dim)
input_tensor = input_tensor.to(device7)
output = model(input_tensor)
print(output.shape)
output[:,:2,:]

torch.Size([4, 864, 1])


tensor([[[0.5076],
         [0.5096]],

        [[0.5075],
         [0.5120]],

        [[0.5070],
         [0.5099]],

        [[0.5105],
         [0.5116]]], device='cuda:7', grad_fn=<SliceBackward0>)

In [26]:

'''
Mean-std normalization function. 
Example input: shape (5000)
Example output: shape (5000)

Used to normalize AccV, AccML, AccAP values.
'''

def sample_normalize(sample):
    # sample: (N,)
    if not isinstance(sample, torch.Tensor):
        sample = torch.tensor(sample)
    mean = torch.mean(sample)
    std = torch.std(sample)
    # Normalize the sample and handle division by zero
    eps = 1e-8
    normalized_sample = (sample - mean) / (std + eps)
    return normalized_sample

'''
Function for splitting a series into blocks. Blocks can overlap. 
How the function works:
Suppose we have a series with AccV, AccML, AccAP columns and len of 50000, that is (50000, 3). 
First, the series is padded so that the final length is divisible by CFG['block_size'] = 15552. Now the series shape is (62208, 3).
Then we get blocks: first block is series[0:15552, :], second block is series[972:16524, :], ... , last block is series[46656:62208, :].

'''

def get_blocks(series, columns):
    series = series.copy()
    series = series[columns]
    
    block_count = math.ceil(series.shape[0] / CFG['block_size'])
    
    padding = block_count * CFG['block_size'] - series.shape[0]
    padding = torch.zeros(size=(padding, series.shape[1]))

    series = torch.cat([series, padding], dim=0)
    
    block_begins = list(range(0, series.shape[0], CFG['block_stride']))
    block_begins = [x for x in block_begins if x + CFG['block_size'] <= series.shape[0]]
    
    blocks = []
    for begin in block_begins:
        values = series[begin:begin + CFG['block_size']]
        blocks.append({'begin': begin,
                       'end': begin + CFG['block_size'],
                       'values': values})
    
    return blocks


warnings.filterwarnings("ignore")
pd.set_option('display.max_colwidth', None)

def folder(path): 
    if not os.path.exists(path): os.makedirs(path)
        
def plot(e, size=(20, 4)):
    plt.figure(figsize=size)
    plt.plot(e)
    plt.show()

In [ ]:
'''
Create train blocks with AccV, AccML, AccAP, StartHesitation, 
Turn, Walking, Valid, Mask columns and save in the directory

'''

save_path = '/kaggle/working/train/tdcsfog'; folder(save_path); 


# tdcsfog_metadata = pd.read_csv('/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/tdcsfog_metadata.csv').set_index('Id')

# blocks_descriptions = []
# for Id in tqdm(tdcsfog_metadata.index, total=len(tdcsfog_metadata.index), desc='Preparing'):
#     series = pd.read_csv(f'/kaggle/input/tlvmc-parkinsons-freezing-gait-prediction/train/tdcsfog/{Id}.csv')
    
#     series['AccV'] = sample_normalize(series['AccV'].values)
#     series['AccML'] = sample_normalize(series['AccML'].values)
#     series['AccAP'] = sample_normalize(series['AccAP'].values)
#     series['Valid'] = 1
#     series['Mask'] = 1

#     blocks = get_blocks(series, ['AccV', 'AccML', 'AccAP', 'StartHesitation', 'Turn', 'Walking', 'Valid', 'Mask'])

#     for block_count, block in enumerate(blocks):
#         fname, values = f'{Id}_{block_count}.npy', block['values']
#         block_description = {}
#         block_description['Id'] = Id
#         block_description['Count'] = block_count
#         block_description['File'] = fname
#         block_description['Path'] = f'{save_path}/{fname}'
#         block_description['Source'] = 'tsfog'
#         block_description['StartHesitation_size'] = np.sum(values[:, 3])
#         block_description['Turn_size'] = np.sum(values[:, 4])
#         block_description['Walking_size'] = np.sum(values[:, 5])
#         block_description['Valid_size'] = np.sum(values[:, 6])
#         block_description['Mask_size'] = np.sum(values[:, 7])

#         blocks_descriptions.append(block_description)
#         np.save(f'{save_path}/{fname}', values)

# blocks_descriptions = pd.DataFrame(blocks_descriptions)